В ноутбуке собирается набор данных для обучения классификаторов над скрытыми состояниями моделей для дальнейшего анализа. Признаками являются векторы скрытых состояний, таргетами типы ошибок.

- Для сбора данных предварительно сгенерируйте векторы скрытых состояний

In [2]:
import os
import sys

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

In [3]:
import pandas as pd

from datasets import load_dataset

In [4]:
ds = load_dataset("ContributorsSIGNAL/SIGNAL")

df = ds["train"].to_pandas()

In [5]:
df.head()

,sentence_id,sentence,congruent,structure,length,target,position,most_popular,percent,semantics_grammar,...,object_gender,object_ipm,gen_lemma,gen_length,gen_gender,gen_ipm,adj_lemma,adj_length,adj_gender,adj_ipm
0,0.0,Автобусы проходят массовую дезинфекцией,Автобусы проходят массовую дезинфекцию,Subject - Verb - Adj - Object,4.0,grammar,3.0,grammar,1.000000,NaN,...,femn,1.63,None,NaN,None,0.02,массовый,4.0,masc,50.834561
1,0.0,Автобусы проходят массовую дезинфекцию,Автобусы проходят массовую дезинфекцию,Subject - Verb - Adj - Object,4.0,normal,3.0,normal,0.833333,NaN,...,femn,1.63,None,NaN,None,0.02,массовый,4.0,masc,50.834561
2,0.0,Автобусы проходят массовую фортуной,Автобусы проходят массовую дезинфекцию,Subject - Verb - Adj - Object,4.0,semantics_grammar,3.0,semantics_grammar,1.000000,1.0,...,femn,1.63,None,NaN,None,0.02,массовый,4.0,masc,50.834561
3,0.0,Автобусы проходят массовую фортуну,Автобусы проходят массовую дезинфекцию,Subject - Verb - Adj - Object,4.0,semantics,3.0,semantics,1.000000,NaN,...,femn,1.63,None,NaN,None,0.02,массовый,4.0,masc,50.834561
4,2.0,Авторы получали подарками,Авторы получали подарки,Subject - Verb - Object,3.0,grammar,2.0,grammar,1.000000,NaN,...,masc,64.02,None,NaN,None,0.02,None,NaN,None,0.020000


In [19]:
targets = df.target

In [7]:
target.head()

0              grammar
1               normal
2    semantics_grammar
3            semantics
4              grammar
Name: target, dtype: object

In [8]:
target

0                grammar
1                 normal
2      semantics_grammar
3              semantics
4                grammar
             ...        
595               normal
596            semantics
597    semantics_grammar
598               normal
599              grammar
Name: target, Length: 600, dtype: object

In [9]:
from src.decoder_utils import calculate_aggregated_hidden_states

In [22]:
import os
import numpy as np

hidden_states_dir = "../persisted_hidden_states"

hidden_states_list: list[tuple[int, np.ndarray]] = []

for base_dir, folders, files in os.walk(hidden_states_dir):
    for file, target in zip(files, targets):
        file_index = int(file.split(".")[0])
        file_path = base_dir + "/" + file
        hidden_states = np.load(file_path)
        hidden_states_list.append((file_index, hidden_states))

In [24]:
hidden_states_list.sort(key=lambda x: x[0])

In [26]:
hidden_states = list(map(lambda x: x[1], hidden_states_list))

In [31]:
d = {
    "hidden": [],
    "layer": [],
    "model": [],
    "target": [],
}

for hidden_state, target in zip(hidden_states, targets):
    for i, layer in enumerate(hidden_state):
        d["hidden"].append(layer)
        d["layer"].append(i)
        d["model"].append("gpt2")
        d["target"].append(target)


In [32]:
import pandas as pd

In [34]:
df = pd.DataFrame(d)

In [35]:
df.head()

,hidden,layer,model,target
0,"[-0.12794064, -0.059372697, 0.18820193, 0.0022...",0,gpt2,grammar
1,"[-1.4979281, -0.73049, 2.0241673, -0.44325504,...",1,gpt2,grammar
2,"[-1.0598001, -0.9965287, 1.6222678, -1.2315387...",2,gpt2,grammar
3,"[-0.9300831, -0.7853385, 1.3130838, -1.1275953...",3,gpt2,grammar
4,"[-1.4587929, -0.517331, 1.4352485, -0.7729259,...",4,gpt2,grammar


In [41]:
probing_dataset = pd.concat([df["hidden"].apply(pd.Series).add_prefix("h_"), df.drop("hidden", axis=1)], axis=1)

In [43]:
probing_dataset.to_csv("../datasets/probing_dataset_gpt2.csv", index=False)